<a href="https://colab.research.google.com/github/oneir0mancer/stable-diffusion-diffusers-colab-ui/blob/main/sd_diffusers_colab_ui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependencies

In [ ]:
!pip install --upgrade diffusers accelerate transformers xformers safetensors
!mkdir -p outputs/{txt2img,img2img}

In [ ]:
!git clone https://github.com/oneir0mancer/stable-diffusion-diffusers-colab-ui.git StableDiffusionUi
!git clone https://huggingface.co/embed/negative /content/embeddings/negative
!git clone https://huggingface.co/embed/lora /content/Lora/positive

In [ ]:
import torch
from diffusers import DiffusionPipeline

output_index = 0
generator = torch.Generator()

# Creating model pipeline
You can use most from [huggingface](https://huggingface.co). Just make sure they are in diffusers format, check their **Files** and see if there is a `model_index.json`.

I made a simple index wih popular models, so you can just render UI and choose a model from dropdown. Some models require trigger word in the prompt.

For weights in Automatic111 format (**.ckpt** or **.safetensors** but without model_index.json), you'll need to download them and convert using scripts (WIP).

In [ ]:
#@title Render model choice UI
from StableDiffusionUi.ColabUI.HugginfaceModelIndex import HugginfaceModelIndex

model_index = HugginfaceModelIndex("/content/StableDiffusionUi/model_index.json")
model_index.render()

In [ ]:
#@title Load chosen model
#@markdown Alternatively you can just paste huggingface model_id or path to model folder here:

path_to_model = ""  #@param {type: "string"}

if path_to_model != "": model_id = path_to_model
else: model_id = model_index.get_model_id()

pipe = DiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")
pipe.safety_checker = None

In [ ]:
#@title (Optional) Change sampler 
from diffusers import EulerAncestralDiscreteScheduler, DPMSolverMultistepScheduler, UniPCMultistepScheduler

choose_sampler = "Euler A" #@param ["Euler A", "DPM++", "DPM++ Karras", "UniPC"]

if choose_sampler == "Euler A":
    solver = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)
elif choose_sampler == "DPM++":
    solver = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
elif choose_sampler == "DPM++ Karras":
    solver = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
    solver.use_karras_sigmas = True
elif choose_sampler == "UniPC":
    solver = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

pipe.scheduler = solver
print(f"Sampler '{choose_sampler}' chosen")

## VAE

Models loaded from huggingface should have proper VAE. But you also can load VAE from any other repository from [huggingface](https://huggingface.co), just paste their **model_id** and subfolder (usually just "vae"). 

Or you can load it in Automatic111 format and convert using scripts.

In [ ]:
#@title (Optional) Load VAE

from diffusers import AutoencoderKL

vae_id_or_path = "runwayml/stable-diffusion-v1-5"  #@param {type: "string"}
vae_subfolder = "vae"    #@param {type: "string"}

vae = AutoencoderKL.from_pretrained("vae_id_or_path", subfolder=vae_subfolder, torch_dtype=torch.float16).to("cuda")
pipe.vae = vae

## Textual inversions and LoRAs
This is work in progress. 

You can load any textual inversions using 
```
pipe.load_textual_inversion("path/to/dir", weight_name="filename.pt")
```
To use them, you need to add theit token to a prompt like this `<filename>`.

Note, that it seems that if you change VAE after loading textual inversions, using them will degrade generated images.

In [ ]:
#@title (Optional) Load textual inversions
#@markdown Load every embedding with **.pt** extension from **embeddings** folder.

import os

for path, subdirs, files in os.walk("/content/embeddings/"):
    for name in files:
        try:
            if os.path.splitext(name)[1] != ".pt": continue
            pipe.load_textual_inversion(path, weight_name=name)
            print(path, name)
        except: pass

# Generating images

In [ ]:
#@title Render UI
#@markdown You don't need to run this cell again unless you want to change these settings
save_images = True #@param {type:"boolean"}
display_previewes = True    #@param {type:"boolean"}

from  StableDiffusionUi.ColabUI.DiffusionPipelineUI import DiffusionPipelineUI

ui = DiffusionPipelineUI()
ui.render()

In [ ]:
#@title Run this cell to generate images
results = ui.generate(pipe, generator)

if display_previewes:
    ui.display_image_previews(results.images)

if save_images:
    for image in results.images:
        image.save(f"outputs/txt2img/{output_index:05}.png")
        print(f"outputs/txt2img/{output_index:05}.png")
        output_index += 1

#Utils

In [ ]:
#@title Image viewer
#@markdown Run this cell to view last results
from IPython.display import clear_output
import ipywidgets as widgets

slider = widgets.IntSlider(max=len(results.images)-1)

def handler(change):
    slider.max = len(results.images)-1
    if change.new > slider.max: change.new = slider.max
    clear_output(wait=True)
    display(slider, results.images[change.new])

slider.observe(handler, names='value')

display(slider, results.images[slider.value])